In [9]:
text_annotation = """
T2	Character 14 23	androcles
A1	Gender T2 Male
T4	Place 69 75	forest
T5	Sound_Effect 86 95	wandering
E3	Sound_Effect:T5 CausedBy:T2 HappensIn:T4
T3	Sound_Effect 57 61	fled
E2	Sound_Effect:T3 CausedBy:T2 HappensIn:T4
T6	Sound_Effect 139 146	moaning
E4	Sound_Effect:T6 CausedBy:T8 HappensIn:T4
T7	Sound_Effect 151 159	groaning
E5	Sound_Effect:T7 CausedBy:T8 HappensIn:T4
T8	Character 123 127	lion
T1	Sound_Effect 512 516	lick
E1	Sound_Effect:T1 CausedBy:T9 HappensIn:T4
T9	Character 477 481	lion
*	RefersToCharacter T9 T8
T10	Sound_Effect 428 438	pulled out
E6	Sound_Effect:T10 CausedBy:T2 HappensIn:T4
T11	Sound_Effect 989 997	bounding
E7	Sound_Effect:T11 CausedBy:T9 HappensIn:T13
T12	Sound_Effect 1002 1009	roaring
E8	Sound_Effect:T12 CausedBy:T9 HappensIn:T13
T13	Place 931 936	arena
T14	Sound_Effect 1119 1125	licked
E9	Sound_Effect:T14 CausedBy:T9 HappensIn:T13 CausedBy2:E10
T15	Sound_Effect 1143 1155	friendly dog
E10	Sound_Effect:T15 
"""

text_text = """
a slave named androcles once escaped from his master and fled to the forest
as he was wandering about there he came upon a lion lying down moaning and groaning
at first he turned to flee but finding that the lion did not pursue him he turned back and went up to him
as he came near the lion put out his paw which was all swollen and bleeding and androcles found that a huge thorn had got into it and was causing all the pain
he pulled out the thorn and bound up the paw of the lion who was soon able to rise and lick the hand of androcles like a dog
then the lion took androcles to his cave and every day used to bring him meat from which to live
but shortly afterwards both androcles and the lion were captured and the slave was sentenced to be thrown to the lion after the latter had been kept without food for several days
the emperor and all his court came to see the spectacle and androcles was led out into the middle of the arena
soon the lion was let loose from his den and rushed bounding and roaring towards his victim
but as soon as he came near to androcles he recognised his friend and fawned upon him and licked his hands like a friendly dog
the emperor surprised at this summoned androcles to him who told him the whole story
whereupon the slave was pardoned and freed and the lion let loose to his native forest
"""

lemmadict = {"a": "a", "slave": "slave", "named": "name", "androcles": "androcle", "once": "once", "escaped": "escape", "from": "from", "his": "he", "master": "master", "and": "and", "fled": "flee", "to": "to", "the": "the", "forest": "forest", "as": "as", "he": "he", "was": "be", "wandering": "wander", "about": "about", "there": "there", "came": "come", "upon": "upon", "lion": "lion", "lying": "lie", "down": "down", "moaning": "moan", "groaning": "groan", "at": "at", "first": "first", "turned": "turn", "flee": "flee", "but": "but", "finding": "find", "that": "that", "did": "do", "not": "not", "pursue": "pursue", "him": "he", "back": "back", "went": "go", "up": "up", "near": "near", "put": "put", "out": "out", "paw": "paw", "which": "which", "all": "all", "swollen": "swollen", "bleeding": "bleeding", "found": "find", "huge": "huge", "thorn": "thorn", "had": "have", "got": "get", "into": "into", "it": "it", "causing": "cause", "pain": "pain", "pulled": "pull", "bound": "bind", "of": "of", "who": "who", "soon": "soon", "able": "able", "rise": "rise", "lick": "lick", "hand": "hand", "like": "like", "dog": "dog", "then": "then", "took": "take", "cave": "cave", "every": "every", "day": "day", "used": "use", "bring": "bring", "meat": "meat", "live": "live", "shortly": "shortly", "afterwards": "afterwards", "both": "both", "were": "be", "captured": "capture", "sentenced": "sentence", "be": "be", "thrown": "throw", "after": "after", "latter": "latter", "been": "be", "kept": "keep", "without": "without", "food": "food", "for": "for", "several": "several", "days": "day", "emperor": "emperor", "court": "court", "see": "see", "spectacle": "spectacle", "led": "lead", "middle": "middle", "arena": "arena", "let": "let", "loose": "loose", "den": "den", "rushed": "rush", "bounding": "bound", "roaring": "roar", "towards": "towards", "victim": "victim", "recognised": "recognise", "friend": "friend", "fawned": "fawn", "licked": "lick", "hands": "hand", "friendly": "friendly", "surprised": "surprise", "this": "this", "summoned": "summon", "told": "tell", "whole": "whole", "story": "story", "whereupon": "whereupon", "pardoned": "pardon", "freed": "free", "native": "native"}

In [3]:
def sentence_no(corpus, char_pos):
    # Number of new lines till the beginning of
    # file.
    
    no_nl = 0
    for n in range(len(corpus)-1, 0, -1):
            if corpus[n] == '\n':
                no_nl += 1
                
    return no_nl

    

In [10]:
sentence_no(text_text, 139)

1

In [76]:
import re
rels = text_annotation.split('\n')
regexp_entity = re.compile('[A-Z][0-9]+\s+[A-Z][A-Za-z_]*\s+[0-9]+\s+[0-9]+\s+[a-z]+')
regexp_relation = re.compile('[A-Z][0-9]+\s+([A-Z][A-Za-z_]*:[A-Z][0-9]+\s*)+')
regexp_transitive = re.compile('\*\s+[A-Z][A-Za-z_]*\s+[A-Z][0-9]+\s[A-Z][0-9]+')

In [29]:

regexp_entity.match(rels[1])

<_sre.SRE_Match object; span=(0, 28), match='T2\tCharacter 14 23\tandrocles'>

In [33]:
rels[5]

'E3\tSound_Effect:T5 CausedBy:T2 HappensIn:T4'

In [51]:
re.match(':',':')
#regexp_relation.match(rels[5])

<_sre.SRE_Match object; span=(0, 1), match=':'>

In [52]:
rels[5]

'E3\tSound_Effect:T5 CausedBy:T2 HappensIn:T4'

In [67]:
regexp_relation.match('E3 Az:T5 ')

<_sre.SRE_Match object; span=(0, 9), match='E3 Az:T5 '>

In [77]:
rels[16]

'*\tRefersToCharacter T9 T8'

In [80]:
regexp_relation.match(rels[16])

In [90]:
re.match('\*\s+[A-Z][A-Za-z_]\s+[A-Z][0-9]+\s+[A-Z][0-9]+','* RefersToCharacter T9 T8')

In [ ]:
relations = ['Sound_Effect', 'Place', 'CausedBy', 'HappensIn', 'CausedBy']